In [1]:
!pip install langchain
!pip install langchain_community
!pip install openai
!pip install docarray
!pip install tiktoken

import os
import openai

from openai import OpenAI
import openai
import os
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
openai.api_key = user_secrets.get_secret("openai_api")
client = OpenAI(
    # This is the default and can be omitted
    api_key=openai.api_key,
)

llm_model = "gpt-3.5-turbo"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.10
    Uninstalling orjson-3.9.10:
      Successfully uninstalled orjson-3.9.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-cv 0.9.0 requires keras

In [2]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.document_loaders import PyPDFLoader

file = '/kaggle/input/how-to-build-a-career-in-ai-pdf/eBook-How-to-Build-a-Career-in-AI.pdf'
loader = PyPDFLoader(file_path=file)
data = loader.load()

In [3]:
from langchain.embeddings import OpenAIEmbeddings
# Initialize the embedding model
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key=openai.api_key)  # Specify the model name

# Create the vector store index
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embedding_model  # Include the embedding model
).from_loaders([loader])

# Initialize the language model
llm = ChatOpenAI(temperature=0.9, model=llm_model, openai_api_key=openai.api_key)

# Create the QA chain
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=index.vectorstore.as_retriever(), 
    verbose=True,
    chain_type_kwargs={
        "document_separator": "<<<<>>>>>"
    }
)


/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
2024-06-09 11:41:08.300437: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-09 11:41:08.300604: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-09 11:41:08.472030: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to regist

In [4]:
data[10]

Document(page_content='PAGE 11\nThe Best Way to Build \na New Habit\nOne of my favorite books is BJ Fogg’s, Tiny Habits: The Small Changes That Change \nEverything. Fogg explains that the best way to build a new habit is to start small \nand succeed, rather than start  too big and fail. For example, rather than trying to \nexercise for 30 minutes a day, he recommends aspiring to do just one push-up, and \ndoing it consistently.\nThis approach may be helpful to those of you who want to spend more time studying. \nIf you start by holding yourself accountable for watching, say, 10 seconds of an \neducational video every day — and you do so consistently — the habit of studying daily \nwill grow naturally. Even if you learn nothing in that 10 seconds, you’re establishing the \nhabit of studying a little every day. On some days, maybe you’ll end up studying for an \nhour or longer.', metadata={'source': '/kaggle/input/how-to-build-a-career-in-ai-pdf/eBook-How-to-Build-a-Career-in-AI.pdf', 'p

In [5]:
data[8]

Document(page_content='PAGE 9In the previous chapter, I introduced three key steps for building a career in AI: learning \nfoundational technical skills, working on projects, and finding a job, all of which is supported \nby being part of a community. In this chapter, I’d like to dive more deeply into the first step: \nlearning foundational skills.\nMore research papers have been published on AI than anyone can read in a lifetime. So, when \nlearning, it’s critical to prioritize topic selection. I believe the most important topics for a technical \ncareer in machine learning are:\nFoundational machine learning skills: For example, it’s important to understand models such \nas linear regression, logistic regression, neural networks, decision trees, clustering, and anomaly \ndetection. Beyond specific models, it’s even more important to understand the core concepts \nbehind how and why machine learning works, such as bias/variance, cost functions, regularization, \noptimization algorithm

In [6]:
examples = [
    {
        "query": "Is machine learning foundations the most important skill?",
        "answer": "Yes"
    },
    {
        "query": "What are Python frameworks you need to learn ?",
        "answer": "Tensorflow and PyTorch"
    }
]

In [7]:
from langchain.evaluation.qa import QAGenerateChain


In [8]:
example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI(model=llm_model, openai_api_key=openai.api_key))


In [9]:
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)
new_examples[0]

/opt/conda/lib/python3.10/site-packages/langchain/chains/llm.py:367: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


{'qa_pairs': {'query': 'According to the document, who is the founder of DeepLearning.AI?',
  'answer': 'The founder of DeepLearning.AI is Andrew Ng, as mentioned in the document.'}}

And so now if we look at what exactly is returned here, we can see a query and we can see an answer. Let’s check the document that this is a question and answer for. 

In [10]:
data[0]


Document(page_content='PAGE 1Founder, DeepLearning.AICollected Insights\nfrom Andrew Ng\nHow to \nBuild\nYour\nCareer\nin AIA Simple Guide\n', metadata={'source': '/kaggle/input/how-to-build-a-career-in-ai-pdf/eBook-How-to-Build-a-Career-in-AI.pdf', 'page': 0})

In [11]:
examples += new_examples


In [12]:
examples

[{'query': 'Is machine learning foundations the most important skill?',
  'answer': 'Yes'},
 {'query': 'What are Python frameworks you need to learn ?',
  'answer': 'Tensorflow and PyTorch'},
 {'qa_pairs': {'query': 'According to the document, who is the founder of DeepLearning.AI?',
   'answer': 'The founder of DeepLearning.AI is Andrew Ng, as mentioned in the document.'}},
 {'qa_pairs': {'query': 'According to Andrew Ng in the document, what is AI compared to and how does he believe it will impact human life?',
   'answer': 'Andrew Ng compares AI to the new electricity and believes that it will transform and improve all areas of human life.'}},
 {'qa_pairs': {'query': 'According to the document, what are the key topics covered in the chapters of the book "How to Build a Career in AI"?',
   'answer': 'The key topics covered in the chapters of the book "How to Build a Career in AI" include: '}},
 {'qa_pairs': {'query': "According to the document, what is the author's comparison between

In [13]:
qa.run(examples[0]["query"])


/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...

> Finished chain.


'Yes, having a strong foundation in machine learning is indeed considered one of the most important skills for a career in AI. Understanding foundational machine learning skills, as well as deep learning concepts and software development, are key components to excel in the field of machine learning.'

In [14]:
import langchain
langchain.debug = True

qa.run(examples[0]["query"])

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Is machine learning foundations the most important skill?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Is machine learning foundations the most important skill?",
  "context": "PAGE 9In the previous chapter, I introduced three key steps for building a career in AI: learning \nfoundational technical skills, working on projects, and finding a job, all of which is supported \nby being part of a community. In this chapter, I’d like to dive more deeply into the first step: \nlearning foundational skills.\nMore research papers have been published on AI than anyone can read in a lifetime. So, when \nlearning, it’s critical to prioritize topic selection. I believe the most important topics for a technical \ncareer in machine learnin

'Yes, foundational machine learning skills are considered crucial for a technical career in AI. Understanding concepts like linear regression, neural networks, decision trees, and core principles behind machine learning is essential for building a career in the field.'

In [15]:
# Turn off the debug mode
langchain.debug = False

In [16]:
# Preprocess examples to ensure 'query' is a top-level key
processed_examples = []

for example in examples:
    if 'qa_pairs' in example:
        processed_examples.append({
            'query': example['qa_pairs']['query'],
            'answer': example['qa_pairs']['answer']
        })
    else:
        processed_examples.append(example)

# Now all examples have a consistent structure
for example in processed_examples:
    print(example)

# Assuming qa is an instance of QAEvalChain or a similar class
predictions = qa.apply(processed_examples)
print(predictions)


{'query': 'Is machine learning foundations the most important skill?', 'answer': 'Yes'}
{'query': 'What are Python frameworks you need to learn ?', 'answer': 'Tensorflow and PyTorch'}
{'query': 'According to the document, who is the founder of DeepLearning.AI?', 'answer': 'The founder of DeepLearning.AI is Andrew Ng, as mentioned in the document.'}
{'query': 'According to Andrew Ng in the document, what is AI compared to and how does he believe it will impact human life?', 'answer': 'Andrew Ng compares AI to the new electricity and believes that it will transform and improve all areas of human life.'}
{'query': 'According to the document, what are the key topics covered in the chapters of the book "How to Build a Career in AI"?', 'answer': 'The key topics covered in the chapters of the book "How to Build a Career in AI" include: '}
{'query': "According to the document, what is the author's comparison between traditional literacy and coding literacy?", 'answer': 'The author compares tra

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.apply` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use batch instead.
  warn_deprecated(



> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.
[{'query': 'Is machine learning foundations the most important skill?', 'answer': 'Yes', 'result': 'Yes, foundational machine learning skills are considered one of the most important skills for a technical career in machine learning. Understanding models like linear regression, logistic regression, neural networks, decision trees, and more, along with core concepts like bias/variance, cost functions, regularization, and optimization algorithms are crucial for success in the field.'}, {'query': 'What are Python frameworks you need to learn ?', 'answer': 'Tensorflow and PyTorch', 'result': 'Some key Python frameworks that are importa

In [17]:
from langchain.evaluation.qa import QAEvalChain
llm = ChatOpenAI(temperature=0, model=llm_model, openai_api_key=openai.api_key)
eval_chain = QAEvalChain.from_llm(llm)
graded_outputs = eval_chain.evaluate(processed_examples, predictions)

In [18]:
for i, eg in enumerate(processed_examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['results'])
    print()

Example 0:
Question: Is machine learning foundations the most important skill?
Real Answer: Yes
Predicted Answer: Yes, foundational machine learning skills are considered one of the most important skills for a technical career in machine learning. Understanding models like linear regression, logistic regression, neural networks, decision trees, and more, along with core concepts like bias/variance, cost functions, regularization, and optimization algorithms are crucial for success in the field.
Predicted Grade: CORRECT

Example 1:
Question: What are Python frameworks you need to learn ?
Real Answer: Tensorflow and PyTorch
Predicted Answer: Some key Python frameworks that are important to learn for machine learning and artificial intelligence development include TensorFlow, PyTorch, and scikit-learn. These frameworks are widely used in the field and provide essential tools and resources for building and training machine learning models.
Predicted Grade: CORRECT

Example 2:
Question: Acc